<a href="https://colab.research.google.com/github/eldercamposds/RegressaoPySpark/blob/main/ClassificacaoPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [4]:
# Criand uma sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Classificação com Spark")\
    .config("spark.ui.port", "4050")\
    .getOrCreate()

# Verificando sessão Spark
print(spark.version)


3.5.5


In [2]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [7]:
churndata = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/PySpark/Spark/download/Churn.csv", header=True, inferSchema=True, sep=";")

In [8]:
churndata.show(3)

+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|      0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1|8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8|1596608|            3|        1|             0|       11393157|     1|
+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
only showing top 3 rows



In [9]:
formula = RFormula(formula="Exited ~ .", featuresCol="features", labelCol="label", handleInvalid="skip")

In [11]:
churn_trans = formula.fit(churndata).transform(churndata).select("features", "label")

In [15]:
churn_trans.show(3, truncate=False)

+--------------------------------------------------------------+-----+
|features                                                      |label|
+--------------------------------------------------------------+-----+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]      |1.0  |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]|0.0  |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]|1.0  |
+--------------------------------------------------------------+-----+
only showing top 3 rows



In [16]:
treino,teste = churn_trans.randomSplit([0.7,0.3], seed=123)